# Nutzungsanalyse Verteilung
Überblick über die Nutzungsart der Gebäude

In [1]:
import pandas as pd
import src.package.charts as charts
import src.package.importer as im
import src.package.consts as c
import seaborn as sns
import matplotlib.pyplot as plt

# set size of seaborn plots
charts.set_preferences(sns)

# Switch between raw and processed dataset
df = im.get_extended_dataset('../package/datasets/full_dataset.csv')
# df = im.get_dataset('../package/datasets/full_dataset.csv', raw=True)
pd.set_option('display.max_rows', 100)  # to show all value_countsö


#### Usage Cluster


In [2]:
print(df[c.FIELD_USAGE_CLUSTER].value_counts())
print("Number of different types: {}".format(len(df[c.FIELD_USAGE_CLUSTER].drop_duplicates().values)))


WOHNEN           286
SCHULEN          190
HALLEN            99
BUERO             96
BETRIEB           65
BEHERBERGUNG      62
GESUNDHEIT        30
KULTUS_KULTUR     27
ANDERES           15
TECHNIK            9
OFFENE_BAUTEN      9
Name: usage_cluster, dtype: int64
Number of different types: 12


#### Hauptnutzung


In [3]:
print(df[c.FIELD_NOM_USAGE_MAIN].value_counts())
print("Number of different types: {}".format(len(df[c.FIELD_NOM_USAGE_MAIN].drop_duplicates().values)))

WOHNBAUTEN__MFH_MEDIUM                                                                139
SCHULBAUTEN__VOLKSSCHULEN                                                              99
FREIZEITBAUTEN_SPORTBAUTEN_ERHOLUNGSBAUTEN__TURNHALLEN_SPORTHALLEN_MEHRZWECKHALLEN     49
SCHULBAUTEN__KINDERGAERTEN_HORTE                                                       35
WOHNBAUTEN__MFH_LOW                                                                    33
                                                                                     ... 
JUSTIZ__WIEDEREINGLIEDERUNGSSTAETTEN                                                    1
VERKEHRSBAUTEN__WERKHOEFE__DIENSTRAUM                                                   1
FREIZEITBAUTEN_SPORTBAUTEN_ERHOLUNGSBAUTEN__GAERTEN_GEWAECHSHAEUSER                     1
VERKEHRSBAUTEN__FLUG__VERWALTUNG                                                        1
GESUNDHEITSBAUTEN__UNIVERSITAETSKLINIKEN_FORSCHUNG                                      1
Name: nom_

## Nutzungen kombinieren
Der Datensatz enthält sehr unterschiedliche Datenmengen pro Nutzungstyp. Wenn zu wenig Daten vorhanden sind, macht das Verwenden des Felds 'usage_cluster' oder weiteres Zusammenfassen Sinn. Für Nutzungstypen mit vielen Daten (Wohnen, Schulen etc.) macht es Sinn, ein spezifischeren Typ zu verwenden.

**Mehrfamilienhäuser**<br>
WOHNBAUTEN__MFH_MEDIUM<br>
WOHNBAUTEN__MFH_LOW<br>
WOHNBAUTEN__MFH_HIGH<br>

In [4]:
types_wohnen_mfh = ['WOHNBAUTEN__MFH_HIGH', 'WOHNBAUTEN__MFH_MEDIUM', 'WOHNBAUTEN__MFH_LOW']
mfh_df = df[df[c.FIELD_NOM_USAGE_MAIN].isin(types_wohnen_mfh)]

charts.lmplot_gf_field(mfh_df, hue=c.FIELD_NOM_USAGE_MAIN)

AttributeError: module 'src.package.charts' has no attribute 'lmplot_gf_hnf'

**Einfamilienhäuser**<br>
WOHNBAUTEN__EFH_REIHEN_LOW<br>
WOHNBAUTEN__EFH_REIHEN_MEDIUM<br>
WOHNBAUTEN__EFH_REIHEN_HIGH

In [ ]:
types_wohnen_efh = ['WOHNBAUTEN__EFH_REIHEN_LOW', 'WOHNBAUTEN__EFH_REIHEN_MEDIUM', 'WOHNBAUTEN__EFH_REIHEN_HIGH']
efh_df = df[df[c.FIELD_NOM_USAGE_MAIN].isin(types_wohnen_efh)]

charts.lmplot_gf_field(efh_df, hue=c.FIELD_NOM_USAGE_MAIN)

**Schulen sinnvoll aufteilen?**<br>
SCHULBAUTEN__VOLKSSCHULEN<br>
SCHULBAUTEN__KINDERGAERTEN_HORTE<br>
SCHULBAUTEN__BERUFSSCHULEN<br>
SCHULBAUTEN__HOCHSCHULEN_UNIVERSITAETEN<br>
SCHULBAUTEN__FORSCHUNGSBAUTEN_MIT_LABOR<br>
SCHULBAUTEN__BIBLIOTHEKEN<br>
SCHULBAUTEN__FORSCHUNGSBAUTEN_OHNE_LABOR<br>
SCHULBAUTEN__HEILPAEDAGOGISCHE_SONDERSCHULEN<br>

In [ ]:
types_schulen = ['SCHULBAUTEN__VOLKSSCHULEN',
                 'SCHULBAUTEN__KINDERGAERTEN_HORTE',
                 'SCHULBAUTEN__BERUFSSCHULEN',
                 'SCHULBAUTEN__HOCHSCHULEN_UNIVERSITAETEN',
                 'SCHULBAUTEN__FORSCHUNGSBAUTEN_MIT_LABOR',
                 'SCHULBAUTEN__BIBLIOTHEKEN',
                 'SCHULBAUTEN__FORSCHUNGSBAUTEN_OHNE_LABOR',
                 'SCHULBAUTEN__HEILPAEDAGOGISCHE_SONDERSCHULEN'
                 ]
schulen_df = df[df[c.FIELD_NOM_USAGE_MAIN].isin(types_schulen)]

charts.lmplot_gf_field(schulen_df, hue=c.FIELD_NOM_USAGE_MAIN)

**Rest**<br>
Vermutlich vorerst auf Usage Cluster zurückgreifen.


